## Importando bibliotecas

In [1]:
import os
import sys
import pandas as pd
import numpy as np

#Plots
import matplotlib.pyplot as plt
import seaborn as sns

# `do not disturb` mode
import warnings                                  
warnings.filterwarnings('ignore')
                           
# Dates Manipulation
from dateutil.relativedelta import relativedelta 
from datetime import datetime, timedelta
import holidays
import calendar

# statistics
import statsmodels.formula.api as smf            
import statsmodels.tsa.api as smt
import statsmodels.api as sm

#Signal Processing
import scipy as sp
import scipy.fftpack
from scipy.optimize import minimize

## Funções e Configurações

In [3]:
#######  function [GRNN]=grnn_v1(p,t,spread) #######
def train_matrix(p,t,spread):
    #Cria Matriz de Treinamento: 
    
    # p -> Matriz contendo os vetores de entrada
    # t -> Matriz contendo os vetores de saída
    # spread -> valor do spread que será utilizado
    
    #Cria matriz com as matrizes de entrada p e t
    GRNN=np.zeros((1+t.shape[0]+p.shape[0],t.shape[1]))
    
    # Verifica o número de padrões utilizados no treinamento "n"
    if p.shape[1] == t.shape[1]: 
        # número de entradas do vetor de entrada "ne"
        GRNN[0,0]=p.shape[0]
        # número de saídas do vetor de saída "ns"
        GRNN[0,1]=t.shape[0]
        # Spread
        GRNN[0,2]=spread
        
        #vetores de entrada P
        GRNN[1:1+p.shape[0],:]=p
        #Vetores de saída T
        GRNN[1+p.shape[0]:1+p.shape[0]+t.shape[0],:]=t
    return GRNN;

In [4]:
####### Função que implementa a GRNN #######

def implementa_grnn(GRNN, X): 
    
    # GRNN - Saída da Função grnn_v1(p,t,spread) #
    # X    - Valor a ser aproximado #

    # Definição de ne, n
    sizep = [GRNN[0,0], GRNN.shape[1]] 
    
    # Definição de ns, n
    sizet = [GRNN[0,1], GRNN.shape[1]] 
    
    # Parâmetro spread
    spread = GRNN[0,2]

    
    p = GRNN[1:1+int(sizep[0]),:]
    t = GRNN[1+int(sizep[0]):1+int(sizep[0])+int(sizet[0]),:]

    c1 = np.zeros((int(sizep[0]),int(sizep[1])))
    c3 = np.zeros((1,int(sizep[1])))
    
    num = np.zeros((int(sizet[0]),int(sizet[1])))
    den = np.copy(c3)
    
    Y = np.zeros((int(sizet[0]),np.size(X,1)))
    A = np.zeros((int(sizep[0]),int(sizep[1])))

    if (p.shape[0] == X.shape[0]):
        for i in range(X.shape[1]):
            for k in range(int(sizep[0])):
                A[k,:] = X[k,i]*np.ones((1,int(sizep[1])))
            c1 = abs(A-p)**2
            # soma "MATRIZ"
            c2 = np.sqrt(c1.sum(axis=0)) 
            
    ########### Aqui é a GRNN ########### 

            for j in range(int(sizep[1])): 
                c3[0,j] = (1)*np.exp(-(0.8326*c2[j]/spread)**2)
                for k in range(int(sizet[0])):
                    num[k,j] = t[k,j]*c3[0,j]
                den[0,j] = c3[0,j]
                
    ########################################


            for k in range(int(sizet[0])):
                #Y(k,i)=sum(num(k,:)/(sum(den)+1e-9)
                Y[k,i]=(np.sum(num[k,:])/(np.sum(den)+0.0000000001))
    return Y

In [5]:
####### Função que implementa a GRNN Modificada #######

def implementa_grnn_mod(GRNN, X, nmax): 
    
    # GRNN - Saída da Função grnn_v1(p,t,spread) #
    # X    - Valor a ser aproximado #
    
    # Definição de ne, n
    sizep=[GRNN[0,0], GRNN.shape[1]]
    
    # Definição de ns, n
    sizet=[GRNN[0,1], GRNN.shape[1]] 
    
    # Parâmetro spread
    spread=GRNN[0,2]

    p=GRNN[1:1+int(sizep[0]),:]
    t=GRNN[1+int(sizep[0]):1+int(sizep[0])+int(sizet[0]),:]

    c1=np.zeros((int(sizep[0]),int(sizep[1])))
    c3=np.zeros((1,nmax))

    num=np.zeros((int(sizet[0]),int(sizet[1])))
    den=np.copy(c3)
    
    Y=np.zeros((int(sizet[0]), X.shape[1]))
    A=np.zeros((int(sizep[0]),int(sizep[1])))
    
    if (p.shape[0] == X.shape[0]):
        for i in range(X.shape[1]):      
            for k in range(int(sizep[0])):
                A[k,:]=X[k,i]*np.ones((1,int(sizep[1])))
            
            c1=abs(A-p)**2
            
            # soma "MATRIZ"
            c2=np.sqrt(c1.sum(axis=0))

########### Aqui é a GRNN modificada ###########
        
            ind_c2 = np.argsort(c2)
            ord_c2 = np.sort(c2)
        
            for j in range(nmax): 
                
                c3[0,j]=(1)*np.exp(-(0.8326*ord_c2[j]/spread)**2)
                
                for k in range(int(sizet[0])):
                    
                    num[k,j]=t[k,ind_c2[j]]*c3[0,j]
                    
                den[0,j]=c3[0,j]
                
####################################################
            
            for k in range(int(sizet[0])):
                Y[k,i]=(np.sum(num[k,:])/(np.sum(den)+0.0000000001))
    return Y


In [6]:
# Vetor de Entrada
p1 = [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]
p = np.array(p1, ndmin = 2)

In [7]:
p2 = [[0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2], [0.4, 0.8, 1.2, 1.6, 2, 2.4, 2.8, 3.2, 3.6, 4]]
p2 = np.array(p2, ndmin = 2)

In [8]:
# Vetor de Saída
t1 = [0.0268, 0.0624, 0.0996, 0.1312, 0.15, 0.1488, 0.1204, 0.0576, -0.0468, -0.2]
t = np.array(t1, ndmin = 2)

In [9]:
spread = 0.1

In [16]:
# Valor a ser aproximado
X1 = [0.5, 0.6, 0.7, 0.8]
X = np.array(X1, ndmin = 2)

x2 = [[0.5], [0.5]]
x2 = np.array(x2, ndmin = 2)

In [20]:
# Matriz de treinamento
W = train_matrix(p,t,spread)
print(W)

[[ 1.      1.      0.1     0.      0.      0.      0.      0.      0.
   0.    ]
 [ 0.2     0.4     0.6     0.8     1.      1.2     1.4     1.6     1.8
   2.    ]
 [ 0.0268  0.0624  0.0996  0.1312  0.15    0.1488  0.1204  0.0576 -0.0468
  -0.2   ]]


In [21]:
# Resultado da GRNN
r_grnn = implementa_grnn(W, X)
print(r_grnn)

[[0.08099222 0.09928868 0.11536422 0.13048841]]


In [23]:
# Resultado da GRNN Modificada
r_m_grnn = implementa_grnn_mod(W, X, 2)
print(r_m_grnn)

[[0.081      0.09741239 0.1154     0.13230557]]
